In [8]:
import os
from langchain_openai import ChatOpenAI
from langgraph.graph import Graph
from langgraph.prebuilt import create_react_agent
import gradio as gr
from datetime import datetime

# Import tools
from tools.calculator import CalculatorTool
from tools.city_recommender import CityRecommendationTool
from tools.email_tool import EmailItineraryTool
from tools.hotel_search import HotelSearchTool
from tools.user_preferences import UserPreferencesTool
from tools.flight_search import FlightSearchTool
from tools.map_tool import MapPlottingTool
from tools.attractions_tool import CityAttractionsTool
from city_vector_db import CityVectorDB

In [9]:
def initialize_tools():

    vector_db = CityVectorDB()
    vector_db.load("cities_vector_db")

    tools = [
        UserPreferencesTool(),
        FlightSearchTool(api_key=os.getenv('SERPAPI_API_KEY')),
        MapPlottingTool(),
        EmailItineraryTool(
            smtp_user="h.belagur1980@gmail.com",
            smtp_password=os.getenv('GMAIL_PASSWORD')
        ),
        CityAttractionsTool(),
        HotelSearchTool(api_key=os.getenv('SERPAPI_API_KEY')),
        CalculatorTool(), 
        CityRecommendationTool(vector_db=vector_db)
    ]
    return tools

In [10]:
tools = initialize_tools()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
system_prompt = f"""You are a helpful travel assistant named Fred.

Follow these steps in order:

1. CITY RECOMMENDATION:
   - Use city_recommendation_tool EXACTLY ONCE to suggest one city
   
2. FLIGHT SEARCH:
   - Use the user's hometown as the departure_airport and the first recommended city as the arrival_airport
   - Ask for specific travel dates (departure and return, if return is not given, get the duration of the trip)
   - Use flight_search_tool with IATA codes
   - Format: {{
       'departure_airport': '[IATA code]',
       'arrival_airport': '[IATA code]',
       'departure_date': 'YYYY-MM-DD',
       'return_date': 'YYYY-MM-DD'  # Optional for one-way flights
   }}

3. HOTEL SEARCH
   - Look for hotels in the recommended city for the dates of the flight (departure and return date) and pick the cheapest one

4. ATTRACTIONS:
   - Once the hotel and flight are selected, use the city_attractions_tool to get the top attractions in the recommended city. Make sure you match the correct formats for the tool

5. CALCULATOR:
   - Use the calculator_tool to sum up the selected flight and hotel costs and also provide the breakdown of the costs   

6. END:
   - Once the user is done with the conversation, thank them and end the conversation
   
Remember that today's date is {datetime.now().strftime('%Y-%m-%d')}. Only search for future dates.
Do not repeat city recommendations unless explicitly asked by the user.

For eg. If the user's name is John Lincoln and they are planning a trip to New York. First, check if John Lincoln has stored preferences using the user_preferences_tool. If not, ask about his travel preferences.
Then using their preferences, recommend a city using the city_recommendation_tool. Ask the user where they are flying from and the dates of travel. Use the flight_search_tool to get the flight details.
Then look for hotels in the destination for the dates of the flight (departure and return date) using the hotel_search_tool. Then look for attractions in the destination using the city_attractions_tool.
Then create a map of the trip using the map_plotting_tool. Then calculate the cost of the trip using the calculator_tool. Then send an email to the user with the itinerary details using the gmail_itinerary_tool 
after the user is done with the conversation asking them for their email.
"""

In [12]:
def create_agent(system_prompt):
    model = ChatOpenAI(model="gpt-4-turbo")
    tools = initialize_tools()
    return create_react_agent(model, tools=tools, state_modifier=system_prompt, debug=True)

In [13]:
agent = create_agent(system_prompt)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
from langchain.callbacks import get_openai_callback

messages = [("user", "my name is sam controlman and i want to plan a holiday from 14th to 15th december. I like skiing and am flying from new york.")]

with get_openai_callback() as cb:
    response = agent.invoke({"messages": messages})
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost:.4f}")

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user',
               'my name is sam controlman and i want to plan a holiday from '
               '14th to 15th december. I like skiing and am flying from new '
               'york.')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user',
  'my name is sam controlman and i want to plan a holiday from 14th to 15th '
  'december. I like skiing and am flying from new york.')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='my name is sam controlman and i want to plan a holiday from 14th to 15th december. I like skiing and am flying from new york.', additional_kwargs={}, response_metadata={}, id='9ec48f79-d56c-4d5d-aa97-76e048cc0175')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='my name is sam controlman and i want to plan a holiday from 1